In [1]:
%run Functions.py

In [30]:
def CheckCoolDownTime(EventTrials_reading, PupilSize_df, NanValues, TimeBaseline, SubjectAndSessionName):

    subjectID = SubjectAndSessionName.split('__')[0]
    timeBaselineCheck = 5 # seconds
    
    pupilSize_baseline = dict()
    pupilSize_baseline['pupilLeft'] = list()
    pupilSize_baseline['pupilRight'] = list()
    
    coolDown = dict()
    coolDown['start'] = list()
    coolDown['end'] = list()
    
    baseline = dict()
    baseline['start'] = list()
    baseline['end'] = list()
    
    for ind, startTimeInd in enumerate(EventTrials_reading['start']):
        startBaselineTime_NextPhraseEnds = PupilSize_df['timeStamp'][startTimeInd]
        
        CoolDownStartTime, CoolDownStartInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), \
                startBaselineTime_NextPhraseEnds-datetime.timedelta(seconds=timeBaselineCheck))
        CoolDownEndTime, CoolDownEndInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), \
                                                          CoolDownStartTime + datetime.timedelta(seconds=timeBaselineCheck))
        
        StartBaselineTime, StartBaselineInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), \
                startBaselineTime_NextPhraseEnds-datetime.timedelta(seconds=TimeBaseline))
        
        EndBaselineTime, EndBaselineInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), StartBaselineTime + \
                                                           datetime.timedelta(seconds=timeBaselineCheck))
        
        
        coolDown['start'].append(CoolDownStartInd)
        coolDown['end'].append(CoolDownEndInd)
        
        
        
        trial_pupilSize = PupilSize_df['timeStamp'][StartBaselineInd:EndBaselineInd]
        trial_nanValues = NanValues[StartBaselineInd:EndBaselineInd]
        
        # find baseline pupil size, for TimeBaseline 
        overlapping_time = 0.1 #s
        sampling_frequency = 90
        nIterations = int((timeBaselineCheck+TimeBaseline)/overlapping_time)+1
        iteration = 0
        done = 0
        while iteration < nIterations:
            NanValuesPresent = trial_nanValues[int(iteration*(TimeBaseline-overlapping_time)*sampling_frequency):\
                                     int((iteration*(TimeBaseline-overlapping_time) + TimeBaseline)*sampling_frequency)]
            
            if sum(NanValuesPresent) < 1 and len(NanValuesPresent)>1:
                #print('done')
                baselineStartInd = StartBaselineInd + int(iteration*(TimeBaseline-overlapping_time)*sampling_frequency)
                baselineEndInd = StartBaselineInd + int((iteration*(TimeBaseline-overlapping_time) + TimeBaseline)*\
                                                        sampling_frequency)
                done = 1
                
                #print(np.mean(PupilSize_df['pupilLeft'][baselineStartInd:baselineEndInd]))
                #print(np.mean(PupilSize_df['pupilRight'][baselineStartInd:baselineEndInd]))
                
                pupilSize_baseline['pupilLeft'].append(np.mean(PupilSize_df['pupilLeft'][baselineStartInd:baselineEndInd]))
                pupilSize_baseline['pupilRight'].append(np.mean(PupilSize_df['pupilRight'][baselineStartInd:baselineEndInd]))
        
                baseline['start'].append(StartBaselineInd)
                baseline['end'].append(EndBaselineInd)
                #plt.figure()
                #plt.plot(PupilSize_df['timeStamp'][baselineStartInd:baselineEndInd], PupilSize_df['pupilLeft'][\
                #    baselineStartInd:baselineEndInd])
                #plt.plot(PupilSize_df['timeStamp'][baselineStartInd:baselineEndInd], PupilSize_df['pupilRight'][\
                #    baselineStartInd:baselineEndInd])
                
                break
            
            iteration = iteration + 1
            
        if done < 1:
            print('baseline not found')
            # add another 5s to the baseline search
            
            extraBaseLineEndTime, extraBaseLineEndInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), EndBaselineTime\
                    +datetime.timedelta(seconds=timeBaselineCheck))
            
            trial_nanValues = NanValues[EndBaselineInd:extraBaseLineEndInd]
            
            nIterations = int((timeBaselineCheck+TimeBaseline)/overlapping_time)+1
            iteration = 0
            done = 0
            while iteration < nIterations:
                NanValuesPresent = trial_nanValues[int(iteration*(TimeBaseline-overlapping_time)*sampling_frequency):\
                                     int((iteration*(TimeBaseline-overlapping_time) + TimeBaseline)*sampling_frequency)]
            
                if sum(NanValuesPresent) < 1 and len(NanValuesPresent)>1:
                    #print('done')
                    baselineStartInd = EndBaselineInd + int(iteration*(TimeBaseline-overlapping_time)*sampling_frequency)
                    baselineEndInd = EndBaselineInd + int((iteration*(TimeBaseline-overlapping_time) + TimeBaseline)*\
                                                        sampling_frequency)
                    done = 1
                
                    #print(np.mean(PupilSize_df['pupilLeft'][baselineStartInd:baselineEndInd]))
                    #print(np.mean(PupilSize_df['pupilRight'][baselineStartInd:baselineEndInd]))
                
                    pupilSize_baseline['pupilLeft'].append(np.mean(PupilSize_df['pupilLeft'][baselineStartInd:baselineEndInd]))
                    pupilSize_baseline['pupilRight'].append(np.mean(PupilSize_df['pupilRight'][baselineStartInd:baselineEndInd]))
                    
                    baseline['start'].append(StartBaselineInd)
                    baseline['end'].append(EndBaselineInd)
                
                    #plt.figure()
                    #plt.plot(PupilSize_df['timeStamp'][baselineStartInd:baselineEndInd], PupilSize_df['pupilLeft'][\
                    #    baselineStartInd:baselineEndInd])
                    #plt.plot(PupilSize_df['timeStamp'][baselineStartInd:baselineEndInd], PupilSize_df['pupilRight'][\
                    #    baselineStartInd:baselineEndInd])
                
                    break
            
                iteration = iteration + 1
            
            if done < 1:
                print('baseline not found -- again')
              
        
    plt.figure()
    plt.plot(PupilSize_df['timeStamp'], PupilSize_df['pupilLeft'], 'cyan')
    
    for i, coolDownStartInd in enumerate(coolDown['start']):
        plt.plot([PupilSize_df['timeStamp'][coolDownStartInd],PupilSize_df['timeStamp'][coolDownStartInd]],[np.min(PupilSize_df['pupilLeft']), np.max(PupilSize_df['pupilLeft'])], color = 'green')
        plt.plot([PupilSize_df['timeStamp'][coolDown['end'][i]],PupilSize_df['timeStamp'][coolDown['end'][i]]],[np.min(PupilSize_df['pupilLeft']), np.max(PupilSize_df['pupilLeft'])], color = 'gray')
        
        plt.plot(PupilSize_df['timeStamp'][baseline['start'][i]], pupilSize_baseline['pupilLeft'][i], '*')
        
        plt.plot([PupilSize_df['timeStamp'][baseline['start'][i]],PupilSize_df['timeStamp'][baseline['start'][i]]],[np.min(PupilSize_df['pupilLeft']), np.max(PupilSize_df['pupilLeft'])], '-.', color = 'green')
        plt.plot([PupilSize_df['timeStamp'][baseline['end'][i]],PupilSize_df['timeStamp'][baseline['end'][i]]],[np.min(PupilSize_df['pupilLeft']), np.max(PupilSize_df['pupilLeft'])], '-.', color = 'gray')
        
    PlotTitle = 'CoolDownAndVaryingBaseline'
    pickle.dump(fig, open(r'C:\DTU\Results\201901_Expt\PupilBaseline' + '\\' + subjectID + '\\' + 'pupilSize_CompleteExperiment_w' + PlotTitle + '_' + SubjectAndSessionName, 'wb'))
    
    
    return pupilSize_baseline

In [2]:
metricComputed_right = 'PupilSizeBaselineRight'
metricComputed_left =  'PupilSizeBaselineLeft'

dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4

resultFileName_right = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Subject_Block_Session_Trial_' + metricComputed_right +  '.xlsx'
resultFileName_left = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Subject_Block_Session_Trial_' + metricComputed_left +  '.xlsx'

baselinePupilSize = dict()
baselinePupilSize['pupilLeft'] = list()
baselinePupilSize['pupilRight'] = list()

baselinePupilSize1 = dict()
baselinePupilSize1['pupilLeft'] = list()
baselinePupilSize1['pupilRight'] = list()

baselinePupilSize2 = dict()
baselinePupilSize2['pupilLeft'] = list()
baselinePupilSize2['pupilRight'] = list()

for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        if 'ac\\' not in root:
            continue
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        #if 'bh2\\3\\2019-02-28-16-47-35_1' not in root:
        #    continue
        
        
        
        #if '2019-1-16-16-36-17_1stPart_2' not in root:
        #    continue
            
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                    
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or phraseLog is None or gazeLog is None:
            continue
        else:
                
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            
            
            # need to fix keys selected, where rows get combined because of an inverted comma
            keysSelected_new = FixKeysSelected(keysSelected)
            
            # find start time of typing
            timeTyping = OptiKeyTypingTime(userKeys_new)
            
            timeBaselineQuestion = getBaselineQuestion(timeTyping, keysSelected_new)
            
            
            
            """
            # divide complete data into epochs of phrases
            timeStartEndDict_reading, timeStartEndDict_writing = FindTrialTimes(keysSelected_new, timeTyping, root)
            
            if sessionFolderName in dict_noGazeData:
                print('no gaze data present')
                baselinePupilSize['pupilRight'] = [np.nan]*len(timeStartEndDict_reading['start'])
                baselinePupilSize['pupilLeft'] = [np.nan]*len(timeStartEndDict_writing['start'])
                
            else:
            
               # filter the data
                pupilData_filtered, nanValues = FilterPupilSize(gazeLog, timeTyping, subjAndSessionName)
            
                # find a vector indicating True for the start and end of trials, False otherwise
                eventTrials_reading, eventTrials_readingIndex = DivideTimeIntoTrials(pupilData_filtered, timeStartEndDict_reading)
                eventTrials_writing, eventTrials_writingIndex = DivideTimeIntoTrials(pupilData_filtered, timeStartEndDict_writing)
            
                # baseline time extraction
                #pupilSize_baseLine = GetBaseline(eventTrials_reading, pupilData_filtered)
                timeBaseline = 0.3 # seconds
                CheckCoolDownTime(eventTrials_readingIndex, pupilData_filtered, nanValues, timeBaseline, subjAndSessionName)
            
            
            if '1stPart' in root:
                print('1stPart')
                baselinePupilSize1['pupilRight'] = baselinePupilSize['pupilRight']
                baselinePupilSize1['pupilLeft'] = baselinePupilSize['pupilLeft']
                
                print(baselinePupilSize1)
                
                continue
                
            if '2ndPart' in root:
                print('2ndPart')
                baselinePupilSize2['pupilRight'] = baselinePupilSize1['pupilRight']
                baselinePupilSize2['pupilLeft'] = baselinePupilSize1['pupilLeft']
                
                baselinePupilSize['pupilRight'] = baselinePupilSize1['pupilRight'] + baselinePupilSize2['pupilRight']
                baselinePupilSize['pupilLeft'] = baselinePupilSize1['pupilLeft'] + baselinePupilSize2['pupilLeft']
                
                baselinePupilSize1['pupilLeft'] = list()
                baselinePupilSize1['pupilRight'] = list()
                
             
            
            # save the ipa_reading
            dataToSave_right = DataForEveryTrial()
            dataToSave_right.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_right.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_right.sessionNumber = subjAndSessionName[-1]
            dataToSave_right.variable = metricComputed_right
            dataToSave_right.dataForTrial = baselinePupilSize['pupilRight']
            dataToSave_right.resultPathName = resultFileName_right
            
            print(dataToSave_right.printInfo())
            dataToSave_right.AddToFile()
            
            
            
            # save the ipa_reading
            dataToSave_left = DataForEveryTrial()
            dataToSave_left.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_left.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_left.sessionNumber = subjAndSessionName[-1]
            dataToSave_left.variable = metricComputed_left
            dataToSave_left.dataForTrial = baselinePupilSize['pupilLeft']
            dataToSave_left.resultPathName = resultFileName_left
            
            print(dataToSave_left.printInfo())
            dataToSave_left.AddToFile()
            
            """

subject path E:\Data\Data\ac\1\2019-02-11-11-18-30_1
subject and session name:  ac__1__2019-02-11-11-18-30_1
2019-02-11 11:18:35.498852 2019-02-11 11:18:42.867362
subject path E:\Data\Data\ac\1\2019-02-11-11-33-32_2
subject and session name:  ac__1__2019-02-11-11-33-32_2
2019-02-11 11:33:43.254479 2019-02-11 11:33:54.564414
subject path E:\Data\Data\ac\2\2019-02-12-15-00-04_1
subject and session name:  ac__2__2019-02-12-15-00-04_1
2019-02-12 15:00:08.926311 2019-02-12 15:00:17.216577
subject path E:\Data\Data\ac\2\2019-02-12-15-27-37_2
subject and session name:  ac__2__2019-02-12-15-27-37_2
2019-02-12 15:27:42.679109 2019-02-12 15:27:49.581126
subject path E:\Data\Data\ac\3_MS\2019-02-14-14-28-49_1
subject and session name:  ac__3_MS__2019-02-14-14-28-49_1
2019-02-14 14:28:53.273463 2019-02-14 14:29:33.208768
subject path E:\Data\Data\ac\3_MS\2019-02-14-14-45-49_2
subject and session name:  ac__3_MS__2019-02-14-14-45-49_2
2019-02-14 14:45:53.753655 2019-02-14 14:46:44.590368
subject pa

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-af74efe17892>", line 136, in <module>
    userKeys_new = FixUserKeys(userKeys)
  File "C:\DTU\PythonScripts\Analysis_Expt_201901\Functions.py", line 286, in FixUserKeys
    range(0, len(UserKeys_ProgressPercent))]
  File "C:\DTU\PythonScripts\Analysis_Expt_201901\Functions.py", line 285, in <listcomp>
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2039, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handlin

KeyboardInterrupt: 